# Xfinity Offer Scraping
#### By Julia Geller


In [1]:
# variables to collect

# address_full
# major_city
# state
# lat
# lon
# collection_datetime
# proivder
# speed_down
# speed_up
# speed_unit
# price
# technology
# package
# fastest_speed_down


In [2]:
import requests
import json
import numpy as np

In [3]:
def get_offer(house_number: str, street_name: str, street_type:str, city: str, state: str, zip_code: str, lat: float, long: float):
    """
    Gets the response from https://www.xfinity.com/learn/landing/active-address for searching for 
    an offer at the given address in the United States.
    Parameters:
        house_number - str house number for the address
        street_name - str street name (i.e. Huntington)
        street_type - str type of street (i.e. Avenue, Boulevard, etc.)
        city - str name of city
        state - str state abberviation (i.e. MA)
        zip_code - str zip code
        lat - float latitude of the address
        long - float longitude for the address
    Return:
    json offer found for the given address
    """
    # https://www.xfinity.com/learn/landing/active-address
    cookies = {
    'PSC': 'UCID=1c421100-42fa-4bd7-8ac4-6f4285dc29de&CTY={0}&ST={1}&Z={2}&EX=False&REC=N&RC.MKT=5091&RC.RLOC=232548872'.format(city.upper(), state.upper(), zip_code),
    '__privaci_cookie_consents': '{"consents":{"77":1,"78":1,"79":1,"80":1},"location":"'+ state.upper()+'#US","lang":"en","gpcInBrowserOnConsent":false,"gpcStatusInPortalOnConsent":false,"status":"record-consent-success","implicit_consent":true}',
    'SC': 'RC.USID=f4c78f88-d973-4999-9d80-38cf98646d69&VA=71681&RC.IFL=Y&RC.LOC=232548872&L1ID=11599&L2ID=5051&L3ID=6132&L4ID=3846&RC.IPL=N&RC.SP=d8400ca2-eb37-4f2d-8b0d-0d4651452e71&Z=[{0}]&CTY={1}&ST={2}&RC.MKT=5091&RC.BIL=CSG&RC.SYS=8773&RC.PRIN=1000&RC.AGT=800&RC.CORP=0&RC.CRPN=0&RC.FRCH=0&RC.FRCHN=800&RC.HK=38285288EA8EE203FA316A90971A6848'.format(zip_code, city.upper(), state.upper()),
    }
    
    response = requests.post('https://digital.xfinity.com/offers/api/offerpackaging', cookies=cookies, headers={}, json={})
    
    return response.json()


In [4]:
def process_offer(offer: json):
    """
    Processes the offer returned from from https://www.xfinity.com/learn/landing/active-address for a specific address.
    Parameters:
        offer - json response from searching for the offer using get_offer()
    Return:
    plan_name - str name of the cheapest internet plan
    price - str price in dollars of the cheapest internet plan
    speed_down - str download speed for the cheapest internet plan
    units_down = str units for the download speed for the cheapest internet plan
    speed_up - str upload speed for the cheapest internet plan
    units_up - str units for the upload speed for the cheapest internet plan
    """
    provider = 'xfinity'
    
    content = offer['CONTENT']

    # get all plans and price for plan
    plan_price = [(content[key]['Name'],content[key]['EdpPrice']) for key in content.keys() if 'InternetPlan' in content[key].keys()]
    # filter out plans that are not full internet service, manually verified
    plan_price = [pp for pp in plan_price if pp[0] not in ['Internet Essentials', 'Internet Essentials Plus']]
    cheapest_plan = plan_price[np.argmin([eval(pp[1]) for pp in plan_price])]

    plan_name = cheapest_plan[0]
    price = cheapest_plan[1]

    # get details for cheapest plan
    plan_details = [content[key] for key in content.keys() if content[key]['Name']==plan_name][0]

    # get upload speed and units
    upload_info = plan_details['UploadSpeed'].split(' ')
    speed_up= upload_info[len(upload_info)-2]
    units_up= upload_info[len(upload_info)-1]

    # get upload speed and units
    download_info = plan_details['DownloadSpeed'].split(' ')
    speed_down = download_info[len(download_info)-2]
    units_down = download_info[len(download_info)-1]

    return provider, plan_name, price,speed_down, units_down, speed_up, units_up

In [5]:
offer = get_offer('1274', 'Beacon', 'Street', 'Newton', 'MA', '02468', None, None)
process_offer(offer)

KeyError: 'CONTENT'

Rostex Scrape Xfinity

In [6]:
class CoverageResult:
	def __init__(self, is_covered, max_upspeed='', max_downspeed=''):
		self.is_covered = is_covered
		self.down_speed = max_downspeed
		self.up_speed = max_upspeed

In [7]:
class Address:
	def __init__(self, firstline, zipcode, city, state, secondline=None, apt_type='', apt_number=''):
		self.line1 = firstline
		self.line2 = secondline
		self.city = city
		self.state = state
		self.zipcode = zipcode
		self.apt_type = apt_type
		self.apt_number = apt_number

	def fullAddressNoZip(self):
		line = self.line1
		if self.city is not None and self.state is not None:
			line = line + ", " + self.city + ", " + self.state
		return line

	def fullAddress(self):
		line = self.line1
		if self.city is not None and self.state is not None:
			line = line + ", " + self.city + ", " + self.state + ", " + self.zipcode
		return line

	def fullAddressNoCommas(self):
		line = self.line1
		if self.city is not None and self.state is not None:
			line = line + " " + self.city + " " + self.state + " " + self.zipcode
		return line

	def fullAddressWithApt(self):
		if self.apt_type == None:
			self.apt_type = ''
		if self.apt_number == None:
			self.apt_number = ''
		line = self.line1
		if self.apt_type != '' and self.apt_number != '':
			line += ", " + self.apt_type + " " + self.apt_number
		elif self.apt_number != '':
			line += ', ' + self.apt_number
		line += ", " + self.city + ", " + self.state + ", " + self.zipcode
		return line

	def fullAddressForCenturyLink(self):
		return

In [8]:
import pprint
def make_request(address, debug = False, retries = 0, proxy=None):

	# All response "buckets"
	NOT_COVERED = 0
	COVERED_GENERAL = 1
	COVERED_EXISTS = 2
	COVERED_NOT_EXISTS = 3
	NOT_RECOGNIZED = 4
	BUSINESS = 5
	COVERED_BUILDING = 6
	BUILDING_UNKNOWN = 7
	UNKNOWN = 30

	RESPONSES = {
	'xfinity' : {
		'SORRY': 20,
		'EXTRAATTENTION': 29,
		'BULKTENANT': 28,
		'COMMUNITY': 21,
	}
}

	pp = pprint.PrettyPrinter(indent=4)

	# ISP: XFinity 
	# This part of the tool used to work, but now gets uniformly rate-limited
	s = requests.session()

	# Get location ID
	url = 'https://www.xfinity.com/learn/api/neptune/address-search/suggestions?search=' + address.line1 + '&showMore=true'
	url = url.replace(' ','+')

	proxies = None

	headers = {
		# 'user-agent':user_agent,
		'accept': 'application/json, text/plain, */*',
		'accept-encoding': 'gzip, deflate, br',
		'accept-language': 'en-US,en;q=0.9',
		'adrum': 'isAjax:false',
		#'cookie': r'ADRUM=s=1589918082820&r=https%3A%2F%2Fwww.xfinity.com%2Flocations%2Fin-my-area%3F0; PSC=UCID=060d1efe-aee4-404c-9847-5df6eb803de1&CTY=Princeton&ST=NJ&Z=08540&RC.MKT=13560; SC=RC.USID=fee8d365-fbce-40ed-8c3f-107856b5d645&VA=1025&CTY=Princeton&ST=NJ&Z=08540&GEO=True&RC.MKT=13560&L1ID=11599&L2ID=1187&L3ID=265&L4ID=13468; ASP.NET_SessionId=wqpvvgfe2peesp12glncctmc; SC_ANALYTICS_GLOBAL_COOKIE=9c39d0546c7a410985b8a7edd3a54581|False; www-prd_ch=NMANJGKC; AKA_A2=A; bm_sz=E82738A22B8AC11B448E0B3E011A997D~YAAQHFoauOgzgilyAQAAcamALgedOc9tEwoLiN/tArl1RoR97YJrKjkIOd6y2K6Kd4nJw1wCZSfMvG+ilL6s8iZvc97yBjWwZCPcjJgKtHywsA+FRdpmRUfEI+cR/mViwKNQEXX8c2W2BaSaNWs/qiyLKGqhHVqE1r09W0QvcWWk35BQGUoSCTr252ZQx4NPLQ==; check=true; AMCVS_DA11332E5321D0550A490D45%40AdobeOrg=1; AMCV_DA11332E5321D0550A490D45%40AdobeOrg=-1712354808%7CMCIDTS%7C18402%7CMCMID%7C51393300678773604611761550336054447001%7CMCAAMLH-1590522884%7C7%7CMCAAMB-1590522884%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y%7CMCOPTOUT-1589925284s%7CNONE%7CMCAID%7C2F2F6A168515F6EA-6000084AA265A365%7CvVersion%7C4.3.0; _abck=59A625B814EB066EE992A33FC778F96A~0~YAAQHFoauGo1gilyAQAAlrKALgP+vnl04VJBjjv/80Lrd54m9zSmQR8g2SVxusJwoPVQeQQcU5QqQ7h6IX4YIFktvOWZ1s7nKVvwXjwtEgGdg4dGlZwlXO6sPt90NtJSRabDpE2W8Uej3hOKQt69zhwfgHvoZ7mqW7Rfy4Wra8EO4qfvUaRcfeFjYYswY41RQorhc9pqFsNaDr5NTx0GcbFFOugfzs4rKklbOI9FDNvSC/b7toNWrckKJD2lty2Mbr1y/K4yj3WIxGNmpc2AYFgA+Rsm4i5TIQtr8HlaqrmZZIzX4JrSR0/XoRi3g+QogatYzqFdKxQ=~-1~-1~-1; ak_bmsc=9688BB6B67DDE8F864C60AD8F51967BBB81A5A1CD96D00008339C45EFAB6EA39~plfNJBJIOvdSuagcdiAmeEvb5PIY9MLD1HX8Qno8mga1qPXJsqycaD92WsvsCKd96Hdy8uNkYRoaBkww5e91tClSCpSCSnJPYF7dMLX0J7cvpmi8s9KOJKeL1ki5IHr/CwUgthl/EUrpdpr0o8NHMjLLUCPU79fdL9ivxlaVek2xuJ/YCxQA9iLvEQicy/Ju1GRaws2PcYcMHG3n3kdtLgXGnpQpJWJFGjRvt6QaIVsGcxJGRAzPSHkQtXUSHrPhBE; s_sess=%20s_ttc_ev87%3D1589918087464%3B%20s_cc%3Dtrue%3B; _uetsid=089569c7-4554-51e4-950d-b27ca42b0ba0; IR_gbd=xfinity.com; IR_8543=1589918089316%7C0%7C1589918089316%7C%7C; _gcl_au=1.1.1979574038.1589918090; _ga=GA1.2.1142565253.1589918090; _gid=GA1.2.1719159832.1589918090; _fbp=fb.1.1589918089793.1453179654; mbox=session#06ba33ad4b464de88230911f6a69c66a#1589919950; _gat=1; AAMC_comcast_0=REGION%7C7; fltk=segID%3D2450016%2CsegID%3D6761408%2CsegID%3D6761294%2CsegID%3D6759340%2CsegID%3D2056160%2CsegID%3D1662899%2CsegID%3D12965809; aam_sitecore=metro%3Dct_new_york%2Cmetro%3Dde_philadelphia%2Cmetro%3Dms_jackson%2Cexisting_customer%3Dtrue%2C1947324%3Dy%2C1898505%3Dy%2C1296524%3Dy%2C1915698%3Dy%2C2064043%3Dy%2C2450016%3Dy%2C2506414%3Dy%2C2506411%3Dy%2C2506406%3Dy%2C2818943%3Dy%2C2802245%3Dy%2C2802247%3Dy%2C2056160%3Dy%2C2064654%3Dy%2C2069065%3Dy%2C2802245%3Dy%2C2802247%3Dy; aam_uuid=51230999057439989541745199821450114878; QuantumMetricUserID=0399238198e0c138f42983890f3957b3; QuantumMetricSessionID=cbb11adf3e58b97eac02c19dace411fc; ipe.22299.pageViewedCount=0; ipe.22299.pageViewedDay=140; ipe_22299_fov={"numberOfVisits":1,"sessionId":"13c2558d-f9e6-0ac5-fbf6-b1354c666417","expiry":"2020-06-18T19:54:58.585Z","lastVisit":"2020-05-19T19:54:58.585Z"}; ipe_v=9f540253-52e5-bff5-169a-d6c0829a31eb; IPE_Campaign=no_campaign; ipe_s=5577a4c8-8e76-adfc-0a88-a77267d8758a; bm_sv=C4E09F3554BBB332F0B56B4D09A8EF60~x3j02Zl5qVTQsFMxUouPfJ2ZuRcRvpDkPRoMeR9q8F06074Q5e3NH1BMgf94jJfwclUxwtiK9ArM08KE98YvfbUyPWzo6kOGM591js/bf7tBWHvJzeA/8ZCnSNTDl5lpAcW/EGctl3qCHA+s+L+r5ybDptQquXn0tn0d4B0zYrw=; s_pers=%20s_dfa%3Dcomcastdotcomprod%7C1589919886289%3B%20gpv_Page%3Dresi%257Csales%257Clocal%257Carea%257Cmy-area%257Cbundles%7C1589919898287%3B%20stc18%3D%7C1589919904481%3B%20s_lv%3D1589918104492%7C1684526104492%3B%20s_lv_s%3DFirst%2520Visit%7C1589919904492%3B; s_sq=comcastdotcomprod%3D%2526c.%2526a.%2526activitymap.%2526page%253Dhttps%25253A%25252F%25252Fwww.xfinity.com%25252Flocations%25252Fin-my-area%2526link%253DContinue%2526region%253DBODY%2526.activitymap%2526.a%2526.c%2526pid%253Dhttps%25253A%25252F%25252Fwww.xfinity.com%25252Flocations%25252Fin-my-area%2526oid%253DContinue%2526oidt%253D3%2526ot%253DSUBMIT',
		'referer': 'https://www.xfinity.com/locations/in-my-area',
		'sec-fetch-dest': 'empty',
		'sec-fetch-mode': 'cors',
		'sec-fetch-site': 'same-origin',
		'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36',
	}
	r = s.get(url = url,headers=headers,proxies=proxies)
	if r.status_code == 403:
		raise Exception("1. Got 403 error when trying to get location")
	response = r.json()

	if debug == True:
		print(url)
		print('ADDRESS SEARCH')
		pp.pprint(response)

	location_id = ''
	apt_type = ''
	apt_number = ''

	for address_info in response.get('Data'):
		# Only take Id if Id exists and the suggested address's street address and zip code are the same as our's
		if address_info.get('Address').get('Address_StreetAddress') == address.line1 and address_info.get('Address').get('Address_ZipCode') == address.zipcode and address_info.get('Address').get('Address_LocationId') != '':
			# Pick the apt of the first suggested address that matches our's (we're just picking a random unit in the building)
			location_id = address_info.get('Address').get('Address_LocationId')
			apt_type = address_info.get('Address').get('Address_UnitType')
			apt_number = address_info.get('Address').get('Address_UnitNumber')
			break

	# Just to make apt number and type consistent (the xfinity API returns None and not '')
	if apt_type == None:
		apt_type = ''
	if apt_number == None:
		apt_number = ''

	if debug == True:
		print('LOCATION ID: ' + location_id)
		print(apt_type)
		print(apt_number)

	url = 'https://www.xfinity.com/learn/api/neptune/localize'

	# Create body and add addresss-specific parameters
	body = {
		"Address": {
			"IsRequestFromSingleAddressField":"true",
			"SalesSessionId": "",
			"SearchReferenceId": "",
			"Over25.unitNumber":"",
		},
		"Address_MDU_Units": "",
		"FullAddress": "",
		"OfferId": "",
		"PageName": "",
		"RedirectURL": "reload",
		"RedirectionType": "",
		"Source": "",
		"TenantInfo" : {
			"Affiliate": "",
			"AgentNTID": "",
			"AvayaAgentId": "",
			"CSGOperatorId": "",
			"CSGSalesRepId": "",
			"Channel": "",
			"ConsumerType": "",
			"DDPOperatorId": "",
			"DDPSalesRepId": "",
			"Flavor": "",
			"Issuer": "",
			"OmniConsumerType": "",
			"StoreId": "",
			"UId": "",
			"UcId": "",
		},
		"idIsSingleInputFallbackSituation": "",
		"idOriginalBeforeFallbackHeader": "",
		"idOriginalBeforeFallbackSubheader": "",
		"isDirectLinking": "False",
	}
	body["Address"]["SingleStreetAddress"] = address.line1 + ", " + address.city + ", " + address.state + " " + address.zipcode
	body["Address"]["StreetAddress"] = address.line1
	body["Address"]['zipcode'] = address.zipcode
	if location_id != '':
		body["Address"]['LocationId'] = location_id
	if apt_type != '':
		body["Address"]["UnitType"] = apt_type
		body["Address"]["UnitNumber"] = apt_number
	else:
		body["Address"]["UnitType"] = ''
		body["Address"]["UnitNumber"] = ''

	# Make request
	headers = {
		'accept': 'application/json, text/plain, */*',
		'accept-encoding': 'gzip, deflate, br',
		'accept-language': 'en-US,en;q=0.9',
		'content-type': 'application/json;charset=UTF-8',
		'cookie': r'ASP.NET_SessionId=pwzeyw5wfegguvz23znrsl2c; SC_ANALYTICS_GLOBAL_COOKIE=74fb545832d14ebd866deb32f67aa292|False; check=true; AMCVS_DA11332E5321D0550A490D45%40AdobeOrg=1; ipe_s=951bc68b-c27f-3bf3-c2de-5234c2b585f9; QuantumMetricUserID=bb3ac11ab236b6985a0767484b607bcf; IR_gbd=xfinity.com; AAMC_comcast_0=REGION%7C7; aam_sc=aamsc%3D8163787; aam_uuid=51230999057439989541745199821450114878; _ga=GA1.2.757503275.1593805680; _gcl_au=1.1.1297193439.1593805680; _fbp=fb.1.1593805680760.979696246; IPE_Campaign=no_campaign; ipe_v=44dd0b16-e4b5-a10f-a22d-155fbdf5f102; aam_sitecore=metro%3Dct_new_york%2Cmetro%3Dde_philadelphia%2Cmetro%3Ddc_washington%2Cmetro%3Dms_jackson%2Cexisting_customer%3Dtrue%2C1898505%3Dy%2C1296524%3Dy%2C2064043%3Dy%2C2450016%3Dy%2C2506406%3Dy%2C2818943%3Dy%2C2802245%3Dy%2C2802247%3Dy%2C2056160%3Dy%2C2745468%3Dy%2C2802245%3Dy%2C2802247%3Dy; QuantumMetricSessionID=9603bb01e5e3109fc9c2947cd850a6a1; www-prd_wc=LEBLFHEE; AKA_A2=A; bm_sz=F637B594C8D8CB5F42CBC010D727732A~YAAQDWAZuPKg/ulzAQAAcCV4+AhfwkjTxg5x38gsYymqs1Ao/jqxkIRci0lYpBPm/PiZ1GdQRB7hbGNIFkGpsuIr5rMQUDwnuArk7rGyqK1grdJU3xR3OPD1sljipffL2LbE0nWxmdqrEGYv8ES2Qp/18bN2fngd6Dq3/R/acILxWCw/ZA7k0LIKn66qTXdD; ak_bmsc=FE40E2A2B743F6163E321AC04D57A9C9B819600DE11E0000D276395FA5C4D73E~plXEi0QTj/3133kPzeEfMBTQkQo/YqrDxAwrhbCkb6OPxccyjM8DvumSUtysLA5zLXhz+mbO5hSJG2SCREE8KaQhKixY1I3FOXzK/CMsaRnJuHLWwlQlskY019aohlq1QEuTNQEd+6ulRKhug9vt6f3CVn9NgWxJEH3BcATDvwLDQmV6sR8R4hAqoMujus8sJtNWXc6lBGjaUzNQ78HLRoprwxetAOTUL40Hxzn38ToOjL2pQd03bYRGf77ioatiN5; ipe.22299.pageViewedDay=229; _gid=GA1.2.1424399153.1597601499; fltk=segID%3D2450016%2CsegID%3D6761408%2CsegID%3D6761294%2CsegID%3D6759340%2CsegID%3D2056160%2CsegID%3D12965809; bounceClientVisit1369v=N4IgNgDiBcIBYBcEQM4FIDMBBNAmAYnvgO6kB0AHgGYCWAdjQgJ5kDGA9gLZFgCmAhgCc6PfnQAm9AOZF2AVwQBadlUVV27ZIPoJM+AMoJBvXgizjxxlOgwARAJwA2PAFZcABgDCAeQByAcQAlAFF9fVcPfQAVPEcsCCEETl46XTssAAUY3Dd3AEZYgC0aCE92cV5MW3z7AHZ3EAAaEEEYEFJiSloGZjYuEABfIA; _abck=BBB26F8A80B86D06DFB960E458193A97~0~YAAQDWAZuNOq/ulzAQAAkv95+AT+9QUvVYQ0GB6Yzjm2GSkcwgKb+SaPg30wznY/Z6ho9taZHuvP3vlmoENF5eQI9ogeaVd0U5W+Q+VixHG4HDaQVStr6DqBytdyjFHv4UtVtghtiCK4V1l7gyHdYM6Yjj+TB9cLDAKpd4PhnmFrQWLgog3ZDiXJOquf+IJeplSXnyn/KNtDNfDpJeYbq9n25fOEXloOoXtVXWKMevl6ai7vsks6Vo0HXNoZXkFdW0CxU2PWg04qiTq+MjeX3/FSx/cCTHnDI22kctQnkdD2LeBiBh6DegAVAFVT9+lzJJ7FLmtCk8g=~-1~-1~-1; SC=RC.USID=7d7fbc47-ffe3-451d-b418-9403b9a4a621&VA=34817&RC.IFL=Y&RC.LOC=233423993&L1ID=11599&L2ID=5051&L3ID=6132&L4ID=3846&RC.SP=699c2449-c2c7-422a-bdd6-9f82f12dc8ee&CTY=METHUEN&ST=MA&RC.MKT=6114&RC.BIL=CSG&RC.SYS=8773&RC.PRIN=1000&RC.AGT=2490&RC.CORP=0&RC.CRPN=0&RC.FRCH=0&RC.FRCHN=2490&RC.HK=1E09EF8851E2840AF9D4414F377B5E6A&Z=01844&RC.IPL=N; PSC=UCID=35b542d7-5a9f-43c8-abde-ea9603408856&CTY=METHUEN&ST=MA&Z=01844&RC.MKT=6114&RC.RLOC=233423993&REC=N&EX=False; s_sess=%20s_ttc_ev87%3D1597601629835%3B%20s_cc%3Dtrue%3B; AMCV_DA11332E5321D0550A490D45%40AdobeOrg=-1712354808%7CMCIDTS%7C18491%7CMCMID%7C51393300678773604611761550336054447001%7CMCAAMLH-1598206430%7C7%7CMCAAMB-1598206430%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y%7CMCOPTOUT-1597608830s%7CNONE%7CMCAID%7C2F2F6A168515F6EA-6000084AA265A365%7CvVersion%7C4.3.0%7CMCCIDH%7C788371218; ADRUM=s=1597601692454&r=https%3A%2F%2Fwww.xfinity.com%2Flearn%2Foffers%3F0; mbox=session#5045eaca453f4da0a72e3792e1827b9b#1597603554; _uetsid=29e13f06f9612236113d2a9ce05d2c07; _uetvid=6b5377263e05e9108fd8882a1cb031c2; IR_8543=1597601695078%7C0%7C1597601494592%7C%7C; _gat=1; ipe.22299.pageViewedCount=9; ipe_22299_fov={"numberOfVisits":1,"sessionId":"951bc68b-c27f-3bf3-c2de-5234c2b585f9","expiry":"2020-09-15T18:11:35.375Z","lastVisit":"2020-08-16T18:14:56.517Z"}; bm_sv=C470A4C989B3DCC3EEF6917A807C6861~VhyFMLTAgrLLjmtS3OYrf/ImvsbBl0AaZku/dbxh/Phueim2Rp+katXETu+CGr04WdN8o6yWXNopDU4WAKWbU6ppmJyRvJ05Cwfuw4yIbmNNHf2FC9UmGrifnGi6+XnrHMXAzrcITDdMwsF8t0qyMZWxwNqsDIVvJIuOELPqnBY=; s_pers=%20s_dfa%3Dcomcastdotcomprod%7C1597603493919%3B%20gpv_Page%3Dresi%257Csales%257Cshop%257Clearn%257Clocalization%257Center%2520address%7C1597603499387%3B%20stc18%3D%7C1597603507661%3B%20s_lv%3D1597601707673%7C1692209707673%3B%20s_lv_s%3DMore%2520than%25207%2520days%7C1597603507673%3B; s_sq=comcastdotcomprod%3D%2526c.%2526a.%2526activitymap.%2526page%253Dresi%25257Csales%25257Cshop%25257Clearn%25257Cout%252520of%252520footprint%2526link%253DContinue%2526region%253DBODY%2526pageIDType%253D1%2526.activitymap%2526.a%2526.c%2526pid%253Dresi%25257Csales%25257Cshop%25257Clearn%25257Cout%252520of%252520footprint%2526pidt%253D1%2526oid%253DContinue%2526oidt%253D3%2526ot%253DSUBMIT',
		'origin': 'https://www.xfinity.com',
		'referer': 'https://www.xfinity.com/locations/in-my-area',
		'sec-fetch-dest': 'empty',
		'sec-fetch-mode': 'cors',
		'sec-fetch-site': 'same-origin',
		'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36',#user_agent,
		#'cookie': 'SC_ANALYTICS_GLOBAL_COOKIE=a68ca012bef54650811790252ef569bb|False; _gcl_au=1.1.999842113.1582320523; _fbp=fb.1.1582320522825.212911589; QuantumMetricUserID=0399238198e0c138f42983890f3957b3; ipe_v=185d0c4f-2efa-898b-dc47-9993cd604f56; AAMC_comcast_0=REGION%7C7; aam_uuid=51230999057439989541745199821450114878; _ga=GA1.2.1683647772.1582320524; fltk=segID%3D2450016%2CsegID%3D6761408%2CsegID%3D6761294%2CsegID%3D6759340%2CsegID%3D2056160%2CsegID%3D12965809; ASP.NET_SessionId=oqmk5fmg3e3ucrrepjnj1sr3; check=true; AMCVS_DA11332E5321D0550A490D45%40AdobeOrg=1; IR_gbd=xfinity.com; ipe_s=5577a4c8-8e76-adfc-0a88-a77267d8758a; IPE_Campaign=no_campaign; aam_sitecore=metro%3Dct_new_york%2Cmetro%3Dde_philadelphia%2Cmetro%3Dms_jackson%2Cexisting_customer%3Dtrue%2C1947324%3Dy%2C1898505%3Dy%2C1296524%3Dy%2C1915698%3Dy%2C2064043%3Dy%2C2450016%3Dy%2C2506414%3Dy%2C2506411%3Dy%2C2506406%3Dy%2C2818943%3Dy%2C2802245%3Dy%2C2802247%3Dy%2C2056160%3Dy%2C2064654%3Dy%2C2069065%3Dy%2C2802245%3Dy%2C2802247%3Dy; PSC=UCID=da2e50bf-f86f-4c00-baa3-fb4b3c941165&CTY=KNOXVILLE&ST=TN&Z=37922&RC.MKT=14063&RC.RLOC=278061894&REC=N&EX=False; pulse_insights_udid=1329323b-1a9b-4249-aa1b-bbc2549b19fa; pi_pageview_count=1; pi_visit_track=true; pi_visit_count=2; ipe_planBuilder=true; _abck=A1D684E19B15AA9BFB030A21BE736BEE~0~YAAQi/NUuFCNxcBxAQAA6mdeFgOTf8aU4aUoe9Oz4d6ihjkkAoE7PTgWEm240UEf/90jvCleYOaJZ88g5G7POELjDUQbI3d5lAkfXhb3LNA3CoYhPAVVNsq512WatLaOxAvLo02WkQAA/Z1jlyllvMj8joN3lWj5zgrBiphWjGUBmrumk4/H+OVZXC1gozpfJyFiaKTw5eRNuu+yEhbvA6lFvG3TL31xPMe3C2RyMm9x2B6HBpXf8juw/kcY9Sbe8DRTMPDVb7JwA35IMpK351NHYHIvyQWiKt/azPZcUlMoE9yFaCIDyjD5UJlzEFjASN0UoQwcQ1c=~-1~-1~-1; ipe.22299.pageViewedCount=1; ADRUM=s=1589520094546&r=https%3A%2F%2Fwww.xfinity.com%2Flocations%2Fin-my-area%3F0; QuantumMetricSessionID=8fdbc3ba4cf2792337b90918bc344cd4; www-prd_wc=LKBLFHEE; bm_sz=849F8C0F2442CF2A23F9A3357AB03CD7~YAAQ0iv2SNAMeylyAQAAa2LMLQcIvrmGexi7Q6dXa99SCEWMNGAPzMhBUPv4u7CdfceSTwenZUfnZlvNIRLM9W2ce0iffH5rnf5GhprKdHUwvZJLxdh15RmF5Vr219uNlHOltUfQkt86Ygzhl59B/1PVu3AYqWUk7p2wZwksQ4GzLezpC8kAeRwSlajnx/MQEg==; AMCV_DA11332E5321D0550A490D45%40AdobeOrg=-1712354808%7CMCIDTS%7C18402%7CMCMID%7C51393300678773604611761550336054447001%7CMCAAMLH-1590511070%7C7%7CMCAAMB-1590511070%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y%7CMCOPTOUT-1589913470s%7CNONE%7CMCAID%7CNONE%7CvVersion%7C4.3.0%7CMCCIDH%7C1931064449; ak_bmsc=E9D30DFBC98CE49BF3E85E7D361D97BD48F62BD2032500005C0BC45E1F26CD5F~plv5EgITuNNhloga2rDqH65X2SLz5X1Qy2yzS3XK+850kMOUeC84QnnXWAno4eUEfh6KEqEUKkVIsw8gwAlp9POfEviac+D+LNKFkeyMJN2ZxhojS3mw96nA0Z9iOxXhaymm9ZD1gZrjg8yloBflKGJj4hGqIIokWUPB0nZKHcq17XOm4pGEFVP36o07OJKne7Ea1Mf5jqs3KF6JXj3I2ie3TB/AUWLKrPFOE10IMyTAXwRm65Qjzq9d5dVdHb4DKQ; _uetsid=998de89c-af56-b502-04f4-06f472e7b9f5; IR_8543=1589906277124%7C0%7C1589906277124%7C%7C; SC=RC.USID=6e107d15-957c-4b2d-9f51-1b15041291ad&VA=34817&RC.IFL=Y&RC.LOC=278061894&L1ID=11592&L2ID=6342&L3ID=6343&L4ID=14042&RC.SP=a314c6b3-a9ab-4bdf-9281-eec56be24ee6&CTY=KNOXVILLE&ST=TN&RC.MKT=14063&RC.BIL=CSG&RC.SYS=8396&RC.PRIN=5000&RC.AGT=40&RC.CORP=0&RC.CRPN=0&RC.FRCH=0&RC.FRCHN=40&RC.HK=5BB1A0D8F39094D0E3266A1DA2597CB8&Z=37922; s_sess=%20s_ttc_ev87%3D1589906277513%3B%20s_cc%3Dtrue%3B; AKA_A2=A; _gid=GA1.2.913473278.1589906279; ipe.22299.pageViewedDay=140; ipe_22299_fov={"numberOfVisits":3,"sessionId":"5577a4c8-8e76-adfc-0a88-a77267d8758a","expiry":"2020-06-12T16:27:53.431Z","lastVisit":"2020-05-19T16:37:59.119Z"}; mbox=session#cad3f6390d2945c29ced5c515490b38d#1589908140; bm_sv=A458F0BC354D3C811072DBFA09DCA94F~ioBOvIVPpT1y2TwqT+y0p0IUKDxL98j71BQjSYgLdWb5gZh7/6H8S7/5X/RUDoGGdhHb7LO/rt0MByliFOu3yzgnj56TnUYjzb8xQme9CBRAvtoUO55bHR4Z9b0zZOVltTPlY1ue66g378qqJXpfYizl7Q4gYTCfLIuZd+sD060=; s_pers=%20s_dfa%3Dcomcastdotcomprod%7C1589908074859%3B%20stc18%3D%7C1589908160608%3B%20s_lv%3D1589906360686%7C1684514360686%3B%20s_lv_s%3DLess%2520than%25207%2520days%7C1589908160686%3B%20gpv_Page%3Dresi%257Csales%257Clocal%257Carea%257Cmy-area%257Cbundles%7C1589908160711%3B; s_sq=comcastdotcomprod%3D%2526c.%2526a.%2526activitymap.%2526page%253Dresi%25257Csales%25257Clocal%25257Carea%25257Cmy-area%25257Cbundles%2526link%253DContinue%2526region%253DBODY%2526pageIDType%253D1%2526.activitymap%2526.a%2526.c%2526pid%253Dresi%25257Csales%25257Clocal%25257Carea%25257Cmy-area%25257Cbundles%2526pidt%253D1%2526oid%253DContinue%2526oidt%253D3%2526ot%253DSUBMIT'
		#'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'
	}

	r = s.post(url = url, data=body, headers=headers,proxies=proxies)

	if r.status_code == 403:
		raise Exception("2. Got 403 error")

	response = r.json()

	if debug == True:
		print('COVERAGE INFO')
		pp.pprint(response)
	data = response.get("Data")

	coverage_result = None

	if response.get('RedirectUrl') != None:
		if 'out-of-footprint' in response.get('RedirectUrl'):
			coverage_result = CoverageResult(is_covered = NOT_COVERED)
		elif 'active-address' in response.get('RedirectUrl'):
			coverage_result = CoverageResult(is_covered = COVERED_EXISTS)
		elif 'in-my-area' in response.get('RedirectUrl'):
			coverage_result = CoverageResult(is_covered = COVERED_NOT_EXISTS)
		elif 'business-location' in response.get('RedirectUrl'):
			coverage_result = CoverageResult(is_covered = BUSINESS)
		# elif 'sorry' in response.get('RedirectUrl'):
			# # redo the request
			# if retries < 2:
			# 	return self.make_request(isp='xfinity', address=address, debug=debug, retries=retries+1,proxy=proxy)
			# else:
			# 	return CoverageResult(is_covered = RESPONSES['xfinity']['SORRY'])
		elif 'extra-attention' in response.get('RedirectUrl'):
			coverage_result = CoverageResult(is_covered = RESPONSES['xfinity']['EXTRAATTENTION']) 
		elif 'bulk-tenant' in response.get('RedirectUrl'):
			return CoverageResult(is_covered = RESPONSES['xfinity']['BULKTENANT'])
		elif 'bulk-master' in response.get('RedirectUrl'):
			return CoverageResult(is_covered = RESPONSES['xfinity']['COMMUNITY'])
		else:
			coverage_result = CoverageResult(is_covered = UNKNOWN)
	else:
		if 'SimilarAddresses' in data and len(data.get('SimilarAddresses')) != 0:

			# This returns several apartments. Let's just choose one 
			entered_address = data.get('EnteredAddress').get('StreetAddress')
			suggested_address = data.get('ApiServiceResult').get('Result').get('CustomerProfile')[0].get('Address').get('StreetAddress1')
			
			# Confirming that the address they are suggesting is actually the same
			if entered_address != suggested_address:
				return CoverageResult(is_covered = UNKNOWN+3)

			apt_type = data.get('ApiServiceResult').get('Result').get('CustomerProfile')[0].get('Address').get('UnitType')
			if apt_type == None:
				apt_type = 'abcd'
			apt_number = data.get('ApiServiceResult').get('Result').get('CustomerProfile')[0].get('Address').get('UnitNumber')
			if apt_number == None:
				apt_number = 'abcd'

			if debug == True:
				print("NEW APT TYPE: " + apt_type)
				print("NEW APT NUMBER: " + apt_number)

			new_address = Address(
				firstline = address.line1,
				city = address.city,
				state = address.state,
				zipcode = address.zipcode,
				apt_type = apt_type,
				apt_number = apt_number
			)

			if retries < 1:
				result = make_request(address=new_address, debug=debug, retries=0)
				return result
			else:
				coverage_result = CoverageResult(is_covered = UNKNOWN+4)
		elif 'ValidationIssues' in data:
			if 'The address you entered could not be recognized.' in data.get('ValidationIssues')[0].get('messages')[0]:
				coverage_result = CoverageResult(is_covered = NOT_RECOGNIZED)
			else:
				coverage_result = CoverageResult(is_covered = UNKNOWN+1)
		elif "EnteredAddress" in data:
			coverage_result = CoverageResult(is_covered = COVERED_NOT_EXISTS)
		else:
			coverage_result = CoverageResult(is_covered = UNKNOWN+2)

	return CoverageResult(is_covered = coverage_result.is_covered)


In [9]:
addr = Address(firstline='10094 DELAWARE BAY RD', zipcode='72834',  city='DARDANELLE', state='AR' )
result = make_request(addr)